Вся инфа по развализму:
* Этнический состав регионов
* Религиозный состав регионов
* ВВП на душу регионов
* Структура бюджетов регионов
* Индикаторы ассимиляции этносов 
* Языковая ситуация в регионах

Визуализация:
* Доля русских на карте России
* Степень доминации 1 этноса в регионе
* Этнический состав элиты региона

# Анализ этнического состава регионов

Источник: [Всероссийская перепись населения (2020)](https://rosstat.gov.ru/vpn/2020/Tom5_Nacionalnyj_sostav_i_vladenie_yazykami)

**Базовые задачи**
- [x] Построить таблицы этносов для каждого региона
- [x] Построить таблицы долей этносов для каждого региона
- [x] Написать функцию для препроцессинга сырых данных
- [x] Добавить возможность коротких имен этносов (еще удалить лишние пробелы в конце)
- [x] Топ-5 и топ-10 этносов для каждого региона
- [x] Найти долю русских в каждом регионе
- [x] Объединенная таблица этносов (городское и сельское население)
- [x] Найти ранг русских в каждом регионе
- [ ] Построить удобочитаемые графические репрезентации этих долей
- [ ] Классифицировать все народы в большие группы
- [ ] Найти долю все "почти русских" 
- [ ] Найти долю всех славян
- [x] Измерить степень доминации этой группы в регионе
- [ ] Этнический состав в разрезе "город-село"
- [ ] Доля проживающих за пределами своих республик
- [x] Найти для каждого народа регионы (топ5-топ10) наиболее массового их проживания (в абсолютном и относительном смыслах)
- [ ] Перейти на работу с картами РФ

**Работа с картой регионов**
- [ ] Доля русских на карте субъектов РФ
- [ ] Ранг русских в доли субъектов на карте РФ
- [ ] 

**Разработка библиотеки для работы с ВПН**
- [ ] Построить класс Region и вообще использовать ООП

## Препроцессинг данных

Надо из сырых данных сделать пригодные к выполнению следующих по порядку задач. 

In [745]:
import pandas as pd

In [746]:
dataframe1 = pd.read_excel('Tom5_tab1_VPN-2020.xlsx', sheet_name=None)

In [747]:
def region_ethnic_structure(df, short_names=False):
    rename_dict={}
    df=df[df.columns[(df.isna()).all()!=True]]
    for i in df.columns:
        if 'НАЦИОНАЛЬНЫЙ СОСТАВ НАСЕЛЕНИЯ' not in i: 
            x=df.loc[:,i][df.loc[:,i].first_valid_index()]
            if x not in ['мужчины', 'женщины']:
                t=x
            else:
                x=f'{t}({x[0]})'
        else:
            x='НАЦИОНАЛЬНЫЙ СОСТАВ НАСЕЛЕНИЯ'
        rename_dict[i]=x
    df=df.rename(columns=rename_dict)
    df=df[df.columns[df.columns.str.contains(r'[Нн]аселени[ея]', case=False)]]
    df=df.set_index('НАЦИОНАЛЬНЫЙ СОСТАВ НАСЕЛЕНИЯ').apply(pd.to_numeric, errors='coerce').fillna(0)
    df=df[df.columns[(df==0).all()!=True]]
#     df=df.iloc[:,:10].dropna().set_index('НАЦИОНАЛЬНЫЙ СОСТАВ НАСЕЛЕНИЯ').apply(pd.to_numeric, errors='coerce').fillna(0)
    df=df[(df==0).all(axis=1)!=True]
    if short_names==True:
        df.index=df.index.str.replace(r" \(.{1,}\)", "", regex=True)
        df.index=df.index.str.replace(r"Крымские татары.{1,}", "Крымские татары", regex=True)
        df.index=list(map(lambda x: x.rstrip().lstrip(), df.index))
    return df

## **Этнический состав каждого региона в абсолютных значениях с длинными (оригинальными) именами этносов и с короткими вариантами.**

In [748]:
region_dict={}
for key in dataframe1.keys():
    region_dict[key]=region_ethnic_structure(dataframe1[key])

In [749]:
region_dict={}
for key in dataframe1.keys():
    region_dict[key]=region_ethnic_structure(dataframe1[key], short_names=True)

## **Относительный (в долях) этнический состав регионов.**

In [750]:
share_dict={key: region_dict[key].T.iloc[:,1:].div(region_dict[key].T['Указавшие национальную принадлежность'], axis=0).T for key in region_dict.keys()}

## **Топ-5 и топ-10 этносов в каждом регионе.**

In [751]:
bad_index=['Указавшие национальную принадлежность',
'Лица, в переписных листах которых национальная принадлежность не указана',
'отказавшиеся отвечать на вопрос о национальной принадлежности',
'Указавшие другие ответы о национальной принадлежности (не перечисленные выше) ',
'Указавшие другие ответы о национальной принадлежности ',
'Указавшие другие ответы о национальной принадлежности',
'Нет национальной принадлежности']

In [752]:
top5_dict={key: share_dict[key][~share_dict[key].index.isin(bad_index)]['Городское и сельское население'].sort_values(ascending=False)[0:5] for key in share_dict}

In [753]:
top10_dict={key: share_dict[key][~share_dict[key].index.isin(bad_index)]['Городское и сельское население'].sort_values(ascending=False)[0:10] for key in share_dict}

In [754]:
top5_dict['г. Москва']

Русские          0.902080
Татары           0.008387
Армяне           0.006762
Украинцы         0.005844
Азербайджанцы    0.003704
Name: Городское и сельское население, dtype: float64

In [755]:
top10_dict['Республика Башкортостан']

Русские     0.374934
Башкиры     0.315203
Татары      0.242098
Марийцы     0.021113
Чуваши      0.019862
Удмурты     0.004260
Украинцы    0.003696
Мордва      0.002725
Узбеки      0.002521
Армяне      0.002268
Name: Городское и сельское население, dtype: float64

In [756]:
top10_dict['Республика Татарстан']

Татары     0.535736
Русские    0.403448
Чуваши     0.023178
Кряшены    0.006453
Удмурты    0.005464
Марийцы    0.004013
Узбеки     0.003835
Таджики    0.003213
Мордва     0.003091
Башкиры    0.002826
Name: Городское и сельское население, dtype: float64

## **Сводная таблица полного населения этносов.**

In [762]:
full_abs=pd.concat(
    {key: region_dict[key]['Городское и сельское население'] for key in region_dict},
    axis=1,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
   
    names=None,
    verify_integrity=False,
    copy=True,
).fillna(0)
full_abs=full_abs[~full_abs.index.isin(bad_index)]
full_abs

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
Все население,147182123.0,1540486.0,1169161.0,1348134.0,2308792.0,927828.0,1069904.0,580976.0,1082458.0,1143224.0,...,995686.0,1004125.0,291705.0,1845165.0,1292944.0,766912.0,136085.0,466609.0,150453.0,47490.0
Абазины,41793.0,4.0,0.0,2.0,7.0,0.0,1.0,0.0,2.0,1.0,...,10.0,5.0,16.0,11.0,2.0,1.0,6.0,9.0,0.0,0.0
Абхазы,8177.0,40.0,29.0,26.0,116.0,24.0,30.0,14.0,29.0,44.0,...,7.0,9.0,8.0,13.0,14.0,18.0,3.0,5.0,1.0,2.0
Аварцы,1012074.0,227.0,107.0,253.0,594.0,336.0,256.0,152.0,111.0,259.0,...,227.0,202.0,149.0,241.0,207.0,81.0,20.0,133.0,52.0,16.0
Андийцы,16758.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Эстонцы,7859.0,13.0,14.0,12.0,32.0,8.0,25.0,4.0,12.0,15.0,...,13.0,4.0,17.0,53.0,32.0,20.0,16.0,19.0,1.0,3.0
Сету,234.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Юкагиры,1802.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1510.0,0.0,5.0,3.0,6.0,0.0,69.0,0.0,0.0,151.0
Якуты,478409.0,18.0,15.0,14.0,20.0,10.0,29.0,10.0,12.0,8.0,...,469348.0,147.0,53.0,675.0,987.0,240.0,212.0,67.0,20.0,47.0


In [763]:
full_shares=pd.concat(
    {key: share_dict[key]['Городское и сельское население'] for key in region_dict},
    axis=1,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
).fillna(0)
full_shares=full_shares[~full_shares.index.isin(bad_index)]
full_shares

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
Абазины,0.000320,0.000003,0.000000,1.593678e-06,3.216869e-06,0.000000,9.966562e-07,0.000000,0.000002,9.743607e-07,...,0.000012,0.000006,0.000061,7.462793e-06,0.000002,0.000001,0.000048,0.000021,0.000000,0.000000
Абхазы,0.000063,0.000031,0.000027,2.071781e-05,5.330811e-05,0.000028,2.989969e-05,0.000030,0.000031,4.287187e-05,...,0.000008,0.000010,0.000030,8.819664e-06,0.000012,0.000025,0.000024,0.000012,0.000007,0.000043
Аварцы,0.007750,0.000175,0.000101,2.016002e-04,2.729743e-04,0.000389,2.551440e-04,0.000321,0.000119,2.523594e-04,...,0.000267,0.000228,0.000564,1.635030e-04,0.000184,0.000112,0.000160,0.000307,0.000372,0.000340
Андийцы,0.000128,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000006,0.000000e+00,0.000000,0.000005,0.000000e+00,...,0.000000,0.000001,0.000000,6.784357e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Арчинцы,0.000013,0.000000,0.000000,7.968388e-07,9.191054e-07,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Эстонцы,0.000060,0.000010,0.000013,9.562065e-06,1.470569e-05,0.000009,2.491641e-05,0.000008,0.000013,1.461541e-05,...,0.000015,0.000005,0.000064,3.595709e-05,0.000028,0.000028,0.000128,0.000044,0.000007,0.000064
Сету,0.000002,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000001,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Юкагиры,0.000014,0.000000,0.000000,7.968388e-07,0.000000e+00,0.000000,9.966562e-07,0.000000,0.000000,0.000000e+00,...,0.001779,0.000000,0.000019,2.035307e-06,0.000005,0.000000,0.000551,0.000000,0.000000,0.003210
Якуты,0.003664,0.000014,0.000014,1.115574e-05,9.191054e-06,0.000012,2.890303e-05,0.000021,0.000013,7.794885e-06,...,0.552976,0.000166,0.000201,4.579441e-04,0.000875,0.000331,0.001694,0.000155,0.000143,0.000999


## Доля русских в регионах

In [764]:
rus_dict1={}
for key in share_dict.keys():
    rus_dict1[key]=share_dict[key].loc['Русские']

In [765]:
rus_dict1['Воронежская область']

Городское и сельское население       0.956442
Городское и сельское население(м)    0.951612
Городское и сельское население(ж)    0.960589
Городское население                  0.959178
Городское население(м)               0.953918
Городское население(ж)               0.963598
Сельское население                   0.950887
Сельское население(м)                0.947090
Сельское население(ж)                0.954292
Name: Русские, dtype: float64

In [766]:
rus_dict2={key:share_dict[key].loc['Русские']['Городское и сельское население'] for key in share_dict.keys()}

In [767]:
pd.DataFrame.from_dict(rus_dict2, orient='index', columns=['Русские'])

,Русские
Российская Федерация,0.808495
Белгородская область,0.952871
Брянская область,0.958688
Владимирская область,0.957556
Воронежская область,0.956442
...,...
Амурская область,0.951743
Магаданская область,0.877068
Сахалинская область,0.912026
Еврейская автономная область,0.956658


## Расселение этносов по России

In [768]:
ethnos_abs={key: full_abs.T[key].sort_values(ascending=False) for key in full_abs.index}

In [769]:
ethnos_abs['Русские']

Российская Федерация          105579179.0
г. Москва                       9074375.0
Московская область              6873903.0
Краснодарский край              5121482.0
г. Санкт-Петербург              4275058.0
                                 ...     
Республика Тыва                   31927.0
Ненецкий автономный округ         26021.0
Чукотский автономный округ        25503.0
Чеченская Республика              18225.0
Республика Ингушетия               3294.0
Name: Русские, Length: 88, dtype: float64

In [770]:
ethnos_shares={key: full_shares.T[key].sort_values(ascending=False) for key in full_shares.index}

In [771]:
ethnos_shares['Чеченцы']

Чеченская Республика         0.964237
Республика Дагестан          0.031689
Республика Ингушетия         0.024922
Российская Федерация         0.012826
Республика Калмыкия          0.010998
                               ...   
Ненецкий автономный округ    0.000080
Новосибирская область        0.000079
Республика Бурятия           0.000066
Республика Мордовия          0.000063
Республика Тыва              0.000016
Name: Чеченцы, Length: 88, dtype: float64

In [772]:
where_ethnos_live=full_abs.iloc[:,1:].div(full_abs.iloc[:,0], axis=0)
where_ethnos_live

,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,Московская область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
Все население,0.010467,0.007944,0.009160,0.015687,0.006304,0.007269,0.003947,0.007355,0.007767,0.057919,...,0.006765,0.006822,0.001982,0.012537,0.008785,0.005211,0.000925,0.003170,0.001022,0.000323
Абазины,0.000096,0.000000,0.000048,0.000167,0.000000,0.000024,0.000000,0.000048,0.000024,0.002273,...,0.000239,0.000120,0.000383,0.000263,0.000048,0.000024,0.000144,0.000215,0.000000,0.000000
Абхазы,0.004892,0.003547,0.003180,0.014186,0.002935,0.003669,0.001712,0.003547,0.005381,0.066039,...,0.000856,0.001101,0.000978,0.001590,0.001712,0.002201,0.000367,0.000611,0.000122,0.000245
Аварцы,0.000224,0.000106,0.000250,0.000587,0.000332,0.000253,0.000150,0.000110,0.000256,0.002567,...,0.000224,0.000200,0.000147,0.000238,0.000205,0.000080,0.000020,0.000131,0.000051,0.000016
Андийцы,0.000000,0.000000,0.000000,0.000000,0.000298,0.000000,0.000000,0.000298,0.000000,0.000418,...,0.000000,0.000060,0.000000,0.000060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Эстонцы,0.001654,0.001781,0.001527,0.004072,0.001018,0.003181,0.000509,0.001527,0.001909,0.025449,...,0.001654,0.000509,0.002163,0.006744,0.004072,0.002545,0.002036,0.002418,0.000127,0.000382
Сету,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.004274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Юкагиры,0.000000,0.000000,0.000555,0.000000,0.000000,0.000555,0.000000,0.000000,0.000000,0.003885,...,0.837958,0.000000,0.002775,0.001665,0.003330,0.000000,0.038291,0.000000,0.000000,0.083796
Якуты,0.000038,0.000031,0.000029,0.000042,0.000021,0.000061,0.000021,0.000025,0.000017,0.000918,...,0.981060,0.000307,0.000111,0.001411,0.002063,0.000502,0.000443,0.000140,0.000042,0.000098


In [773]:
where_ethnos_live.loc['Чеченцы'].sort_values(ascending=False)[:10]*100

Чеченская Республика     86.980238
Республика Дагестан       5.930069
Ростовская область        0.854761
Ставропольский край       0.822699
Республика Ингушетия      0.730810
г. Москва                 0.686090
Тюменская область         0.608292
Волгоградская область     0.479922
ХМАО                      0.423022
Астраханская область      0.410364
Name: Чеченцы, dtype: float64

In [774]:
where_ethnos_live.loc['Буряты'].sort_values(ascending=False)[:10]*100

Республика Бурятия          64.182388
Иркутская область           16.247258
Забайкальский край          14.257053
Республика Саха (Якутия)     1.428531
г. Москва                    0.634927
г. Санкт-Петербург           0.344743
Московская область           0.313877
Новосибирская область        0.298444
Красноярский край            0.258014
Хабаровский край             0.248450
Name: Буряты, dtype: float64

In [775]:
where_ethnos_live.loc['Евреи'].sort_values(ascending=False)[:10]*100

г. Москва               33.897198
г. Санкт-Петербург      11.138135
Московская область       6.158947
Свердловская область     2.841102
Самарская область        2.734621
Республика Татарстан     2.167126
Ростовская область       2.040076
Челябинская область      2.026765
Республика Крым          2.002565
Краснодарский край       1.944485
Name: Евреи, dtype: float64

In [776]:
where_ethnos_live.loc['Башкиры'].sort_values(ascending=False)[:10]*100

Республика Башкортостан    80.719063
Челябинская область         8.147637
Тюменская область           2.470228
Оренбургская область        2.301768
ХМАО                        1.890540
Свердловская область        1.099703
Пермский край               1.024570
Республика Татарстан        0.701644
Курганская область          0.510599
ЯНАО                        0.388770
Name: Башкиры, dtype: float64

In [778]:
where_ethnos_live.loc['Цыгане'].sort_values(ascending=False)[:10]*100

Ставропольский край                21.940600
Ростовская область                  8.434256
Краснодарский край                  6.683968
Тамбовская область                  3.050750
Воронежская область                 2.997116
Ульяновская область                 2.503460
Пензенская область                  2.463668
Кабардино-Балкарская Республика     2.377163
Астраханская область                2.277970
Волгоградская область               2.094579
Name: Цыгане, dtype: float64

In [779]:
where_ethnos_live.loc['Татары'].sort_values(ascending=False)[:10]*100

Республика Татарстан        44.364061
Республика Башкортостан     20.674617
Тюменская область            4.141805
Челябинская область          2.550922
Оренбургская область         2.473763
Ульяновская область          2.362597
Тюменская область без АО     2.049189
Самарская область            2.003789
Свердловская область         1.983932
Пермский край                1.961784
Name: Татары, dtype: float64

In [780]:
where_ethnos_live.loc['Таджики'].sort_values(ascending=False)[:10]*100

Московская область      10.775877
Тюменская область        8.506264
г. Москва                6.505042
Свердловская область     6.225231
ХМАО                     6.221805
Калужская область        5.343540
Красноярский край        3.702646
Самарская область        3.667242
Республика Татарстан     3.580728
Челябинская область      3.514202
Name: Таджики, dtype: float64

In [781]:
where_ethnos_live.loc['Армяне'].sort_values(ascending=False)[:10]*100

Краснодарский край       22.314336
Ставропольский край      14.308604
Ростовская область        9.117264
Московская область        7.419264
г. Москва                 7.188756
Самарская область         2.085456
Саратовская область       1.951548
Волгоградская область     1.679504
Республика Адыгея         1.565255
г. Санкт-Петербург        1.557539
Name: Армяне, dtype: float64

## Ранг этносов в каждом регионе

In [783]:
ethnos_rank={key: full_shares.rank(ascending=False).loc[key].sort_values() for key in full_shares.index}

In [784]:
ethnos_rank['Цыгане'][:10]

Тамбовская область      2.0
Новгородская область    3.0
Ставропольский край     4.0
Псковская область       4.0
Курская область         4.0
Пензенская область      4.0
Воронежская область     4.0
Брянская область        4.0
Костромская область     5.0
Липецкая область        5.0
Name: Цыгане, dtype: float64

In [785]:
ethnos_rank['Марийцы'][:10]

Республика Марий Эл           2.0
Кировская область             3.0
Республика Башкортостан       4.0
Удмуртская Республика         4.0
Свердловская область          5.0
Чувашская Республика          5.0
Республика Татарстан          6.0
Нижегородская область        10.0
Ненецкий автономный округ    10.0
Республика Коми              12.0
Name: Марийцы, dtype: float64

In [786]:
ethnos_rank['Украинцы'].value_counts()/ethnos_rank['Украинцы'].value_counts().sum()*100

2.0     22.727273
3.0     18.181818
4.0     13.636364
5.0     12.500000
6.0      9.090909
7.0      7.954545
8.0      2.272727
10.0     2.272727
13.0     2.272727
14.0     2.272727
9.0      1.136364
11.0     1.136364
12.0     1.136364
20.0     1.136364
24.0     1.136364
29.0     1.136364
Name: Украинцы, dtype: float64

In [567]:
full_shares.rank(ascending=False).loc['Русские'][full_shares.rank(ascending=False).loc['Русские']!=1]

Республика Калмыкия                2.0
Республика Дагестан                8.0
Республика Ингушетия               3.0
Кабардино-Балкарская Республика    2.0
Карачаево-Черкесская Республика    2.0
РСО-Алания                         2.0
Чеченская Республика               2.0
Республика Татарстан               2.0
Чувашская Республика               2.0
Республика Тыва                    2.0
Республика Саха (Якутия)           2.0
Name: Русские, dtype: float64

## Степень доминации/разнообразия в регионах

Индекс Херфиндаля-Хиршмана

In [787]:
(full_shares**2).sum(axis=0).sort_values(ascending=False)[-40:]

Саратовская область                0.805927
Самарская область                  0.804548
Пермский край                      0.801714
Забайкальский край                 0.801604
Республика Тыва                    0.796906
Омская область                     0.789391
Камчатский край                    0.780762
Пензенская область                 0.771693
Магаданская область                0.770474
Республика Карелия                 0.750104
Челябинская область                0.749302
Тюменская область без АО           0.742171
Республика Хакасия                 0.691060
Ставропольский край                0.690620
Российская Федерация               0.655742
Оренбургская область               0.637801
Ульяновская область                0.616587
Тюменская область                  0.589388
Республика Крым                    0.558950
Республика Коми                    0.535628
Ненецкий автономный округ          0.520680
Удмуртская Республика              0.519765
Республика Бурятия              

In [788]:
(((full_shares-(full_shares).mean())**2).sum()/88).sort_values(ascending=False)[-10:]

Республика Мордовия                0.005035
Республика Марий Эл                0.004932
Республика Алтай                   0.004823
Республика Саха (Якутия)           0.004644
ЯНАО                               0.004596
Кабардино-Балкарская Республика    0.004318
Чукотский автономный округ         0.004225
Республика Башкортостан            0.003344
Карачаево-Черкесская Республика    0.003317
Республика Дагестан                0.001892
dtype: float64

In [647]:
import numpy as np
pd.DataFrame.from_dict({x:(np.log(1/full_shares[x][full_shares[x]!=0])*full_shares[x][full_shares[x]!=0]).sum()/len(full_shares[x][full_shares[x]!=0]) for x in full_shares.columns}, orient='index',columns=['Энтропия']).sort_values(by='Энтропия',ascending=False)[-30:]
# (np.log(full_shares)*full_shares).sum()

,Энтропия
Краснодарский край,0.002683
Чеченская Республика,0.002562
Смоленская область,0.002544
Орловская область,0.002459
Новгородская область,0.002367
Тульская область,0.002333
Московская область,0.002326
г. Москва,0.002311
Ленинградская область,0.002299
Амурская область,0.002297


## Классификации этносов в широкие группы

In [790]:
families_reverse={'Абазины': 'абхазо-адыгская',
 'Абхазы': 'абхазо-адыгская',
 'Аварцы': 'нахско-дагестанская',
 'Андийцы': 'нахско-дагестанская',
 'Арчинцы': 'нахско-дагестанская',
 'Ахвахцы': 'нахско-дагестанская',
 'Багулалы': 'нахско-дагестанская',
 'Бежтинцы': 'нахско-дагестанская',
 'Ботлихцы': 'нахско-дагестанская',
 'Гинухцы': 'нахско-дагестанская',
 'Годоберинцы': 'нахско-дагестанская',
 'Гунзибцы': 'нахско-дагестанская',
 'Дидойцы': 'нахско-дагестанская',
 'Каратинцы': 'нахско-дагестанская',
 'Тиндалы': 'нахско-дагестанская',
 'Хваршины': 'нахско-дагестанская',
 'Чамалалы': 'нахско-дагестанская',
 'Агулы': 'нахско-дагестанская',
 'Адыгейцы': 'абхазо-адыгская',
 'Азербайджанцы': 'тюркская',
 'Алеуты': 'эскимосско-алеутская',
 'Алтайцы': 'тюркская',
 'Теленгиты': 'тюркская',
 'Тубалары': 'тюркская',
 'Челканцы': 'тюркская',
 'Американцы': 'германская',
 'Арабы': 'семитская',
 'Армяне': 'армянская',
 'Черкесогаи': 'армянская',
 'Ассирийцы': 'семитская',
 'Афганцы': 'индоиранская',
 'Балкарцы': 'тюркская',
 'Башкиры': 'тюркская',
 'Белорусы': 'восточные славяне',
 'Бесермяне': 'финно-угорская',
 'Болгары': 'южные славяне',
 'Боснийцы': 'южные славяне',
 'Британцы': 'германская',
 'Буряты': 'монгольская',
 'Венгры': 'финно-угорская',
 'Вепсы': 'финно-угорская',
 'Водь': 'финно-угорская',
 'Вьетнамцы': 'австроазиатская',
 'Гагаузы': 'тюркская',
 'Горские евреи': 'индоиранская',
 'Греки': 'палеобалканская',
 'Греки-урумы': 'тюркская',
 'Грузинские евреи': 'картвельская',
 'Грузины': 'картвельская',
 'Аджарцы': 'картвельская',
 'Ингилойцы': 'картвельская',
 'Лазы': 'картвельская',
 'Мегрелы': 'картвельская',
 'Сваны': 'картвельская',
 'Даргинцы': 'нахско-дагестанская',
 'Кайтагцы': 'нахско-дагестанская',
 'Кубачинцы': 'нахско-дагестанская',
 'Долганы': 'тюркская',
 'Дунгане': 'сино-тибетская',
 'Евреи': 'семитская',
 'Езиды': 'западноиранская',
 'Ижорцы': 'финно-угорская',
 'Ингуши': 'нахско-дагестанская',
 'Индийцы': 'индоиранская',
 'Испанцы': 'романская',
 'Итальянцы': 'романская',
 'Ительмены': 'чукотско-камчатская',
 'Кабардинцы': 'абхазо-адыгская',
 'Казахи': 'тюркская',
 'Калмыки': 'монгольская',
 'Камчадалы': 'восточные славяне',
 'Караимы': 'тюркская',
 'Каракалпаки': 'тюркская',
 'Карачаевцы': 'тюркская',
 'Карелы': 'финно-угорская',
 'Кереки': 'чукотско-корякская',
 'Кеты': 'енисейская',
 'Юги': 'енисейская',
 'Киргизы': 'тюркская',
 'Китайцы': 'сино-тибетская',
 'Коми': 'финно-угорская',
 'Коми-ижемцы': 'финно-угорская',
 'Коми-пермяки': 'финно-угорская',
 'Корейцы': 'корейская',
 'Коряки': 'чукотско-корякская',
 'Алюторцы': 'чукотско-корякская',
 'Крымские татары': 'тюркская',
 'Крымчаки': 'тюркская',
 'Кубинцы': 'романская',
 'Кумандинцы': 'тюркская',
 'Кумыки': 'тюркская',
 'Курды': 'западноиранская',
 'Курманч': 'западноиранская',
 'Лакцы': 'нахско-дагестанская',
 'Латыши': 'балтийская',
 'Латгальцы': 'балтийская',
 'Лезгины': 'нахско-дагестанская',
 'Литовцы': 'балтийская',
 'Македонцы': 'южные славяне',
 'Манси': 'финно-угорская',
 'Марийцы': 'финно-угорская',
 'Горные марийцы': 'финно-угорская',
 'Лугово-восточные марийцы': 'финно-угорская',
 'Молдаване': 'романская',
 'Монголы': 'монгольская',
 'Мордва': 'финно-угорская',
 'Мордва-мокша': 'финно-угорская',
 'Мордва-эрзя': 'финно-угорская',
 'Нагайбаки': 'тюркская',
 'Нанайцы': 'тунгусо-маньчжурская',
 'Нганасаны': 'самодийская',
 'Негидальцы': 'тунгусо-маньчжурская',
 'Немцы': 'германская',
 'Меннониты': 'германская',
 'Ненцы': 'самодийская',
 'Нивхи': 'ни́вхская',
 'Ногайцы': 'тюркская',
 'Карагаши': 'тюркская',
 'Орочи': 'тунгусо-маньчжурская',
 'Осетины': 'индоиранская',
 'Осетины-дигорцы': 'индоиранская',
 'Осетины-иронцы': 'индоиранская',
 'Пакистанцы': 'индоиранская',
 'Памирцы': 'индоиранская',
 'Персы': 'индоиранская',
 'Поляки': 'западные славяне',
 'Румыны': 'романская',
 'Русины': 'восточные славяне',
 'Русские': 'восточные славяне',
 'Казаки': 'восточные славяне',
 'Поморы': 'финно-угорская',
 'Рутульцы': 'нахско-дагестанская',
 'Саамы': 'финно-угорская',
 'Селькупы': 'самодийская',
 'Сербы': 'южные славяне',
 'Словаки': 'западные славяне',
 'Словенцы': 'южные славяне',
 'Сойоты': 'монгольская',
 'Среднеазиатские евреи': 'индоиранская',
 'Табасараны': 'нахско-дагестанская',
 'Таджики': 'индоиранская',
 'Тазы': 'сино-тибетская',
 'Талыши': 'индоиранская',
 'Татары': 'тюркская',
 'Астраханские татары': 'тюркская',
 'Кряшены': 'тюркская',
 'Мишари': 'тюркская',
 'Сибирские татары': 'тюркская',
 'Таты': 'иранская',
 'Телеуты': 'тюркская',
 'Тофалары': 'тюркская',
 'Тувинцы': 'тюркская',
 'Тувинцы-тоджинцы': 'тюркская',
 'Турки': 'тюркская',
 'Турки-месхетинцы': 'тюркская',
 'Туркмены': 'тюркская',
 'Удины': 'нахско-дагестанская',
 'Удмурты': 'финно-угорская',
 'Удэгейцы': 'тунгусо-маньчжурская',
 'Узбеки': 'тюркская',
 'Уйгуры': 'тюркская',
 'Уйльта': 'тунгусо-маньчжурская',
 'Украинцы': 'восточные славяне',
 'Ульчи': 'тунгусо-маньчжурская',
 'Финны': 'финно-угорская',
 'Финны-ингерманландцы': 'финно-угорская',
 'Французы': 'романская',
 'Хакасы': 'тюркская',
 'Ханты': 'финно-угорская',
 'Хемшилы': 'армянская',
 'Хорваты': 'южные славяне',
 'Цахуры': 'нахско-дагестанская',
 'Цыгане': 'индоиранская',
 'Цыгане среднеазиатские': 'индоиранская',
 'Черкесы': 'абхазо-адыгская',
 'Черногорцы': 'южные славяне',
 'Чехи': 'западные славяне',
 'Чеченцы': 'нахско-дагестанская',
 'Чеченцы-аккинцы': 'нахско-дагестанская',
 'Чуванцы': 'чукотско-камчатская',
 'Чуваши': 'тюркская',
 'Чукчи': 'чукотско-камчатская',
 'Чулымцы': 'тюркская',
 'Шапсуги': 'абхазо-адыгская',
 'Шорцы': 'тюркская',
 'Эвенки': 'тунгусо-маньчжурская',
 'Эвены': 'тунгусо-маньчжурская',
 'Энцы': 'самодийская',
 'Эскимосы': 'эскалеутская',
 'Эстонцы': 'финно-угорская',
 'Сету': 'финно-угорская',
 'Юкагиры': 'юкагирская',
 'Якуты': 'тюркская',
 'Японцы': 'японо-рюкюская'}

In [791]:
families={'абхазо-адыгская': ['Абазины',
  'Абхазы',
  'Адыгейцы',
  'Кабардинцы',
  'Черкесы',
  'Шапсуги'],
 'нахско-дагестанская': ['Аварцы',
  'Андийцы',
  'Арчинцы',
  'Ахвахцы',
  'Багулалы',
  'Бежтинцы',
  'Ботлихцы',
  'Гинухцы',
  'Годоберинцы',
  'Гунзибцы',
  'Дидойцы',
  'Каратинцы',
  'Тиндалы',
  'Хваршины',
  'Чамалалы',
  'Агулы',
  'Даргинцы',
  'Кайтагцы',
  'Кубачинцы',
  'Ингуши',
  'Лакцы',
  'Лезгины',
  'Рутульцы',
  'Табасараны',
  'Удины',
  'Цахуры',
  'Чеченцы',
  'Чеченцы-аккинцы'],
 'тюркская': ['Азербайджанцы',
  'Алтайцы',
  'Теленгиты',
  'Тубалары',
  'Челканцы',
  'Балкарцы',
  'Башкиры',
  'Гагаузы',
  'Греки-урумы',
  'Долганы',
  'Казахи',
  'Караимы',
  'Каракалпаки',
  'Карачаевцы',
  'Киргизы',
  'Крымские татары',
  'Крымчаки',
  'Кумандинцы',
  'Кумыки',
  'Нагайбаки',
  'Ногайцы',
  'Карагаши',
  'Татары',
  'Астраханские татары',
  'Кряшены',
  'Мишари',
  'Сибирские татары',
  'Телеуты',
  'Тофалары',
  'Тувинцы',
  'Тувинцы-тоджинцы',
  'Турки',
  'Турки-месхетинцы',
  'Туркмены',
  'Узбеки',
  'Уйгуры',
  'Хакасы',
  'Чуваши',
  'Чулымцы',
  'Шорцы',
  'Якуты'],
 'эскимосско-алеутская': ['Алеуты'],
 'германская': ['Американцы', 'Британцы', 'Немцы', 'Меннониты'],
 'семитская': ['Арабы', 'Ассирийцы', 'Евреи'],
 'армянская': ['Армяне', 'Черкесогаи', 'Хемшилы'],
 'индоиранская': ['Афганцы',
  'Горские евреи',
  'Индийцы',
  'Осетины',
  'Осетины-дигорцы',
  'Осетины-иронцы',
  'Пакистанцы',
  'Памирцы',
  'Персы',
  'Среднеазиатские евреи',
  'Таджики',
  'Талыши',
  'Цыгане',
  'Цыгане среднеазиатские'],
 'восточные славяне': ['Белорусы',
  'Камчадалы',
  'Русины',
  'Русские',
  'Казаки',
  'Украинцы'],
 'финно-угорская': ['Бесермяне',
  'Венгры',
  'Вепсы',
  'Водь',
  'Ижорцы',
  'Карелы',
  'Коми',
  'Коми-ижемцы',
  'Коми-пермяки',
  'Манси',
  'Марийцы',
  'Горные марийцы',
  'Лугово-восточные марийцы',
  'Мордва',
  'Мордва-мокша',
  'Мордва-эрзя',
  'Поморы',
  'Саамы',
  'Удмурты',
  'Финны',
  'Финны-ингерманландцы',
  'Ханты',
  'Эстонцы',
  'Сету'],
 'южные славяне': ['Болгары',
  'Боснийцы',
  'Македонцы',
  'Сербы',
  'Словенцы',
  'Хорваты',
  'Черногорцы'],
 'монгольская': ['Буряты', 'Калмыки', 'Монголы', 'Сойоты'],
 'австроазиатская': ['Вьетнамцы'],
 'палеобалканская': ['Греки'],
 'картвельская': ['Грузинские евреи',
  'Грузины',
  'Аджарцы',
  'Ингилойцы',
  'Лазы',
  'Мегрелы',
  'Сваны'],
 'сино-тибетская': ['Дунгане', 'Китайцы', 'Тазы'],
 'западноиранская': ['Езиды', 'Курды', 'Курманч'],
 'романская': ['Испанцы',
  'Итальянцы',
  'Кубинцы',
  'Молдаване',
  'Румыны',
  'Французы'],
 'чукотско-камчатская': ['Ительмены', 'Чуванцы', 'Чукчи'],
 'чукотско-корякская': ['Кереки', 'Коряки', 'Алюторцы'],
 'енисейская': ['Кеты', 'Юги'],
 'корейская': ['Корейцы'],
 'балтийская': ['Латыши', 'Латгальцы', 'Литовцы'],
 'тунгусо-маньчжурская': ['Нанайцы',
  'Негидальцы',
  'Орочи',
  'Удэгейцы',
  'Уйльта',
  'Ульчи',
  'Эвенки',
  'Эвены'],
 'самодийская': ['Нганасаны', 'Ненцы', 'Селькупы', 'Энцы'],
 'ни́вхская': ['Нивхи'],
 'западные славяне': ['Поляки', 'Словаки', 'Чехи'],
 'иранская': ['Таты'],
 'эскалеутская': ['Эскимосы'],
 'юкагирская': ['Юкагиры'],
 'японо-рюкюская': ['Японцы']}

In [792]:
full_shares.join(pd.Series(families_reverse, name='Группы')).groupby('Группы').sum()

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
Группы,,,,,,,,,,,,,,,,,,,,,
абхазо-адыгская,0.006137,6.947725e-05,8.900040e-05,5.657555e-05,1.314321e-04,0.000110,1.036522e-04,0.000076,0.000072,6.820525e-05,...,0.000106,0.000108,0.000310,0.000100,0.000106,0.000070,0.000399,0.000178,0.000136,0.000786
австроазиатская,0.000061,6.175756e-06,3.787251e-06,2.996114e-04,1.300534e-04,0.000060,1.255787e-04,0.000085,0.000017,3.897443e-06,...,0.000026,0.000055,0.000000,0.000229,0.000111,0.000041,0.000000,0.000002,0.000000,0.000000
армянская,0.007262,3.806582e-03,3.314792e-03,3.532386e-03,5.172266e-03,0.004018,1.247814e-02,0.002573,0.005574,6.459037e-03,...,0.004600,0.002993,0.003177,0.002970,0.002007,0.004122,0.002093,0.002317,0.001153,0.002062
балтийская,0.000169,6.407347e-05,5.112789e-05,6.295026e-05,6.479693e-05,0.000081,1.225887e-04,0.000089,0.000069,6.820525e-05,...,0.000139,0.000064,0.000182,0.000105,0.000106,0.000115,0.000471,0.000159,0.000186,0.000276
восточные славяне,0.817260,9.657408e-01,9.656553e-01,9.622267e-01,9.632229e-01,0.964007,9.145218e-01,0.967151,0.966223,9.714347e-01,...,0.335936,0.894747,0.905302,0.952995,0.936207,0.959577,0.908157,0.922454,0.967261,0.578395
германская,0.001513,6.206635e-04,1.884157e-04,2.796904e-04,3.166318e-04,0.000294,6.906828e-04,0.000397,0.000270,5.583087e-04,...,0.000772,0.000276,0.000776,0.000419,0.000544,0.000520,0.001111,0.000432,0.000587,0.001190
енисейская,0.000008,0.000000e+00,0.000000e+00,7.968388e-07,4.595527e-07,0.000000,0.000000e+00,0.000000,0.000001,0.000000e+00,...,0.000006,0.000000,0.000000,0.000003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000043
западноиранская,0.000390,2.887166e-04,7.479821e-05,1.059796e-04,5.771982e-04,0.000197,9.866897e-05,0.000135,0.000798,1.230618e-03,...,0.000057,0.000002,0.000310,0.000058,0.000016,0.000017,0.000008,0.000012,0.000086,0.000000
западные славяне,0.000179,1.397265e-04,6.533008e-05,1.298847e-04,8.731501e-05,0.000099,1.624550e-04,0.000125,0.000075,8.184630e-05,...,0.000134,0.000062,0.000231,0.000103,0.000145,0.000102,0.000296,0.000150,0.000200,0.000298


In [793]:
full_groups=full_shares.join(pd.Series(families_reverse, name='Группы')).groupby('Группы').sum()

In [794]:
(full_groups**2).sum(axis=0).sort_values(ascending=False)[-10:]

Республика Татарстан               0.498310
Республика Адыгея                  0.493184
ЯНАО                               0.488472
Республика Башкортостан            0.485160
Республика Марий Эл                0.464612
Республика Калмыкия                0.463567
Республика Саха (Якутия)           0.456658
Кабардино-Балкарская Республика    0.429249
Чукотский автономный округ         0.426288
Карачаево-Черкесская Республика    0.363682
dtype: float64

In [795]:
full_abs_group=full_abs.join(pd.Series(families_reverse, name='Группы')).groupby('Группы').sum()

In [796]:
where_group_live=full_abs_group.iloc[:,1:].div(full_abs_group.iloc[:,0], axis=0)
where_group_live

,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,Московская область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
Группы,,,,,,,,,,,,,,,,,,,,,
абхазо-адыгская,0.000112,0.000117,0.000089,0.000357,0.000119,0.000130,0.000045,0.000084,0.000087,0.003712,...,0.000112,0.000120,0.000102,0.000185,0.000150,0.000064,0.000062,0.000096,0.000024,0.000046
австроазиатская,0.001010,0.000505,0.047487,0.035741,0.006567,0.015913,0.005052,0.002021,0.000505,0.084744,...,0.002778,0.006188,0.000000,0.042561,0.015787,0.003789,0.000000,0.000126,0.000000,0.000000
армянская,0.005200,0.003692,0.004675,0.011869,0.003656,0.013203,0.001284,0.005491,0.006991,0.074036,...,0.004117,0.002796,0.000885,0.004617,0.002386,0.003151,0.000276,0.001058,0.000170,0.000102
балтийская,0.003753,0.002442,0.003572,0.006375,0.003165,0.005561,0.001899,0.002894,0.003165,0.039110,...,0.005335,0.002577,0.002170,0.007008,0.005380,0.003753,0.002668,0.003120,0.001176,0.000588
восточные славяне,0.011722,0.009556,0.011315,0.019639,0.007794,0.008598,0.004289,0.008457,0.009342,0.065073,...,0.002672,0.007425,0.002240,0.013162,0.009893,0.006517,0.001065,0.003742,0.001266,0.000255
германская,0.004070,0.001007,0.001777,0.003488,0.001286,0.003508,0.000952,0.001276,0.002900,0.015089,...,0.003315,0.001235,0.001038,0.003128,0.003108,0.001908,0.000704,0.000947,0.000415,0.000283
енисейская,0.000000,0.000000,0.000913,0.000913,0.000000,0.000000,0.000000,0.000913,0.000000,0.002740,...,0.004566,0.000000,0.000000,0.003653,0.000000,0.000000,0.000000,0.000000,0.000000,0.001826
западноиранская,0.007345,0.001552,0.002612,0.024667,0.003339,0.001944,0.001257,0.014631,0.024805,0.012373,...,0.000943,0.000039,0.001610,0.001669,0.000354,0.000236,0.000020,0.000098,0.000236,0.000000
западные славяне,0.007725,0.002945,0.006957,0.008109,0.003628,0.006957,0.002518,0.002987,0.003585,0.047544,...,0.004865,0.002347,0.002603,0.006487,0.006957,0.003158,0.001579,0.002774,0.001195,0.000597


In [797]:
where_group_live.loc['восточные славяне'].sort_values(ascending=False)[-10:]

РСО-Алания                    0.001157
Магаданская область           0.001065
Республика Алтай              0.001001
Республика Дагестан           0.000965
Республика Калмыкия           0.000621
Республика Тыва               0.000300
Чукотский автономный округ    0.000255
Ненецкий автономный округ     0.000250
Чеченская Республика          0.000173
Республика Ингушетия          0.000031
Name: восточные славяне, dtype: float64

## Универсальный классификатор языков

In [803]:
language_classes={}

In [802]:
"""Абазины
Северокавказская надсемья (необщепризнано)
Абхазо-адыгская семья
Абхазо-абазинская ветвь языков

Абхазы
Северокавказская надсемья (необщепризнано)
Абхазо-адыгская семья
Абхазо-абазинская группа

Аварцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская группа

Андийцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская группа

Арчинцы
кавказские языки
нахско-дагестанские языки
дагестанские языки
лезгинские языки
арчинский язык

Ахвахцы
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская группа

Багулалы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская подгруппа

Бежтинцы
Северокавказская семья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Цезская группа

Ботлихцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская подгруппа

Гинухцы
Северокавказская семья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Цезская группа

Годоберинцы
Северокавказская семья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская группа

Гунзибцы
Северокавказская надсемья (необщепризнана)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Цезская группа

Дидойцы
Северокавказская семья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Цезская группа

Каратинцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская подгруппа

Тиндалы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская подгруппа

Хваршины
Северокавказская семья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Цезская группа

Чамалалы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Аваро-андо-цезская ветвь
Аваро-андийская подветвь
Андийская подгруппа

Агулы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Лезгинская ветвь

Адыгейцы
Северокавказская надсемья (необщепризнано)
Абхазо-адыгская семья
Адыгская группа

Азербайджанцы
Тюркские языки
Юго-западная (огузская) группа
Огузо-сельджукская подгруппа

Алеуты
Эскимосско-алеутская семья
Алеутская ветвь

Алтайцы
Алтайские языки (спорно)
Тюркская ветвь

Теленгиты
Алтайские языки (спорно)
Тюркская ветвь
Горно-алтайская (центрально-восточная) группа
Киргизско-кыпчакская подгруппа
Южноалтайский язык

Тубалары
Алтайская семья
Тюркская ветвь
Горно-алтайская (центрально-восточная) группа
Северноалтайская подгруппа

Челканцы
Алтайская семья
Тюркская ветвь
Горно-алтайская (центрально-восточная) группа
Северноалтайская подгруппа
Северноалтайский язык

Американцы
Индоевропейская семья
Германская ветвь
Западногерманская группа
Англо-фризская подгруппа

Арабы
афразийская макросемья
семитская семья
западносемитская ветвь
центральносемитская группа
аравийская подгруппа

Армяне
Индоевропейская семья
Армянская языковая группа
Восточноармянский и западноармянский

Черкесогаи
Индоевропейская семья
Армянская языковая группа
Восточноармянский и западноармянский

Ассирийцы
Афразийская макросемья
Семитская семья
Западносемитская ветвь
Арамейская группа

Афганцы
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Восточноиранская подгруппа
Юго-Восточноиранская подгруппа

Балкарцы
Тюркская семья
Кыпчакская ветвь
Половецко-кыпчакская группа

Башкиры
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Белорусы
Индоевропейская семья
Славянская группа
Восточнославянская подгруппа

Бесермяне
Уральская семья
Финно-угорская ветвь
Финно-пермская подветвь
Пермская группа

Болгары
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Восточная подгруппа

Боснийцы
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Западная подгруппа

Британцы
Индоевропейская семья
Германская ветвь
Западногерманская группа
Англо-фризская подгруппа

Буряты
Алтайская семья
Монгольская ветвь
Северомонгольская группа
Центральномонгольская подгруппа

Венгры
Уральские языки
Финно-угорские языки
Угорские языки

Вепсы
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Водь
Уральская семья
Финно-угорская ветвь
Прибалтийско-финская группа
Южная подгруппа

Вьетнамцы
Австроазиатская семья
Мон-кхмерская ветвь
Вьетская группа
Вьет-мыонгская подгруппа

Гагаузы
Тюркские языки
Огузская группа

Горские евреи
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Западная подгруппа

Греки
Индоевропейская семья
Палеобалканская ветвь
Греко-фригийско-армянская группа
Греческая группа

Греки-урумы
Алтайские языки (спорно)
Тюркская подветвь
Кыпчакская группа
Половецко-кыпчакская подгруппа
Огузская группа
Турецкая подгруппа

Грузинские евреи
Кавказские языки
Картвельская семья
Грузинский язык
Иудео-грузинский язык

Грузины
Картвельская семья

Аджарцы
Картвельская семья

Ингилойцы
Картвельская семья

Лазы
Картвельская семья

Мегрелы
Картвельская семья
Южнокартвельская ветвь
Занская группа

Сваны
Картвельская семья

Даргинцы
Северокавказская надсемья
Нахско-дагестанская семья

Кайтагцы
Северокавказская надсемья
Нахско-дагестанская семья
Лакско-даргинская надветвь
Даргинская ветвь

Кубачинцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Даргинская ветвь
Кубачи-аштинская группа

Долганы
Алтайские языки (спорно)
Тюркская ветвь
Якутская группа

Дунгане
Сино-тибетские языки
Китайские языки

Евреи
Афразийская макросемья
Семитская семья
Западносемитская группа
Центральносемитская подветвь
Северозападносемитская надгруппа

Езиды
Индоевропейская семья
Арийская ветвь
Иранская подветвь
Западноиранские языки
Северо-западные языки
Курдская подгруппа
Курдский язык

Ижорцы
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Ингуши
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Нахская ветвь
Вайнахская группа

Индийцы
Индоевропейская семья
Индоиранские языки
Индоарийская группа
Центральная подгруппа

Испанцы
Индоевропейская семья
Италийские языки
Романские языки
Западно-романские языки
Иберо-романские языки
Западно-Иберийские языки

Итальянцы
Индоевропейская семья
Италийские языки
Романские языки
Итало-романские языки
Итальянский язык

Ительмены
Чукотско-камчатская семья
Ительменская ветвь

Кабардинцы
Северокавказские языки
Абхазо-адыгская семья
Адыгская группа

Казахи
Алтайские языки
Тюркская ветвь
Кыпчакская группа
Кыпчакско-ногайская подгруппа

Калмыки
Алтайская семья
Монгольская ветвь
Северомонгольская группа
Западномонгольская подгруппа

Камчадалы
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

Караимы
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Половецко-кыпчакская подгруппа

Каракалпаки
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Кыпчакско-ногайская подгруппа

Карачаевцы
Тюркская семья
Кыпчакская ветвь
Половецко-кыпчакская группа

Карелы
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Кереки
Чукотско-камчатская семья
Чукотско-корякская ветвь

Кеты
Енисейская семья
Кетско-югская группа

Юги
Енисейская семья
Кетско-югская группа

Киргизы
Алтайские языки
Тюркская ветвь
Кыпчакская либо горно-алтайская группа
Киргизско-кыпчакская подгруппа

Китайцы
сино-австронезийские языки
сино-тибетские языки
китайский язык

Коми
Уральская семья
Финно-угорская ветвь
Пермская группа

Коми-ижемцы
Уральская семья
Финно-угорская ветвь
Пермская группа

Коми-пермяки
Уральская семья
Финно-угорская ветвь
Пермская группа

Корейцы
Изолированный

Коряки
Чукотско-камчатская семья
Чукотско-корякская ветвь

Алюторцы
Чукотско-камчатская семья
Чукотско-корякская ветвь

Крымские татары
Алтайская семья
Тюркская ветвь
Кыпчакская группа
Половецко-кыпчакская подгруппа (средний диалект и литературный стандарт)
Кыпчакско-ногайская подгруппа (степной диалект)
Огузская группа
Турецкая подгруппа (южнобережный диалект)

Крымчаки
Алтайские языки (спорно)
Тюркская подветвь
Кыпчакская группа
Половецко-кыпчакская подгруппа

Кубинцы
Индоевропейская семья
Италийские языки
Романские языки
Западно-романские языки
Иберо-романские языки
Западно-Иберийские языки

Кумандинцы
Алтайская семья
Тюркская ветвь
Горно-алтайская (центрально-восточная) группа
Северноалтайская подгруппа
Северноалтайский язык

Кумыки
Тюркские языки
Кыпчакская группа
Половецко-кыпчакская подгруппа

Курды
Индоевропейская семья
Арийская ветвь
Иранская подветвь
Западноиранские языки
Северо-западные языки
Курдская подгруппа
Курдский язык

Курманч
Индоевропейская семья
Арийская ветвь
Иранская подветвь
Западноиранские языки
Северо-западные языки
Курдская подгруппа
Курдский язык

Лакцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья

Латыши
Индоевропейская семья
Балто-славянская ветвь (гипотеза)
Балтийская группа
Восточнобалтийская группа

Латгальцы
Индоевропейская семья
Балтийская группа

Лезгины
Северокавказская семья
Нахско-дагестанская группа
Лезгинская подгруппа

Литовцы
Индоевропейская семья
Балто-славянская ветвь
Балтийская группа
Восточнобалтийская подгруппа

Македонцы
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Восточная подгруппа

Манси
Уральская семья
Финно-угорская ветвь
Угорская подветвь
Обско-угорская группа

Марийцы
Уральская семья
Финно-угорская ветвь
Финно-пермские языки

Горные марийцы
Уральская семья
Финно-угорская ветвь
Финно-пермские языки
Марийский язык

Лугово-восточные марийцы
Уральская семья
Финно-угорская ветвь
Финно-пермские языки
Марийский язык

Молдаване
Индоевропейская семья
Италийские языки
Романские языки
Балкано-романские языки
Румынский язык

Монголы
Алтайская семья
Монгольская ветвь
Северомонгольская группа
Центральномонгольская подгруппа

Мордва
Уральская семья

Финно-угорская ветвь
Финно-волжская группа
Мордовская подгруппа

Мордва-мокша
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Мордовская подгруппа

Мордва-эрзя
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Мордовская подгруппа

Нагайбаки
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Нанайцы
Тунгусо-маньчжурская семья
Нанайская группа

Нганасаны
Уральская семья
Самодийская ветвь
Северная группа

Негидальцы
Алтайская семья (спорно)
Тунгусо-маньчжурская ветвь

Немцы
Индоевропейская семья
Германская ветвь
Западногерманская группа
Верхненемецкий
Немецкий язык

Меннониты
Индоевропейская семья
Германская ветвь
Западногерманская группа
Верхненемецкий
Немецкий язык

Ненцы
Уральская семья
Самодийская ветвь
Северная группа

Нивхи
Изолированный язык

Ногайцы
Тюркская ветвь
Кыпчакская группа
Кыпчакско-ногайская подгруппа

Карагаши
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Татарский язык

Орочи
Алтайская семья
Тунгусо-маньчжурская ветвь
Южная группа

Осетины
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Восточно-иранская подгруппа

Осетины-дигорцы
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Восточно-иранская подгруппа

Осетины-иронцы
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Восточно-иранская подгруппа

Пакистанцы
Индоевропейская семья
Индоиранская ветвь
Индоарийская группа
Северо-западная группа

Памирцы
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Юго-восточная подгруппа
Памирские языки

Персы
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Юго-западная подгруппа

Поляки
Индоевропейская семья
Славянская ветвь
Западнославянская группа
Лехитская подгруппа

Румыны
Индоевропейская семья
Италийские языки
Романские языки
Балкано-романские языки
Румынский язык

Русины
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

Русские
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

Казаки
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

Поморы
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

Рутульцы
Северокавказская надсемья
Нахско-дагестанская семья
Лезгинская ветвь

Саамы
Уральская семья
Финно-угорская ветвь
Финно-пермская подветвь
Финно-волжская группа

Селькупы
Уральская семья
Самодийская группа

Сербы
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Западная подгруппа

Словаки
Индоевропейские языки
Славянская ветвь
Западнославянская группа
Чешско-словацкая подгруппа

Словенцы
Индоевропейские языки
Славянские языки
Южнославянские языки
Западная группа

Сойоты
Алтайские языки (спорно)
Тюркские языки
Саянская группа
Таёжный ареал

Среднеазиатские евреи
Индоевропейская семья
Индоиранская (арийская) ветвь
Иранская группа
Юго-западная подгруппа
Таджикский язык
Северные диалекты таджикского языка

Табасараны
Нахско-дагестанская семья
Лезгинская группа

Таджики
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Юго-западная подгруппа

Тазы
Сино-тибетская семья
Китайская ветвь
Севернокитайский язык

Талыши
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Северо-западная подгруппа

Татары
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Астраханские татары
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Кряшены
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Мишари
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Сибирские татары
Алтайские языки (спорно)
Тюркская ветвь
Кыпчакская группа
Поволжско-кыпчакская подгруппа

Таты
Индоевропейская семья
Иранская группа
Западная подгруппа

Телеуты
Алтайские языки (спорно)
Тюркская ветвь
Горно-алтайская (центрально-восточная) группа
Киргизско-кыпчакская подгруппа
Южноалтайский язык

Тофалары
Алтайские языки (спорно)
Тюркские языки
Саянская группа
Таёжный ареал

Тувинцы
Тюркские языки
Саянская группа

Тувинцы-тоджинцы
Тюркские языки
Саянская группа

Турки
Тюркские языки
Юго-западная (огузская) группа
Огузо-сельджукская подгруппа
Османский язык

Турки-месхетинцы
Тюркские языки
Юго-западная (огузская) группа
Огузо-сельджукская подгруппа
Османский язык

Туркмены
Тюркские языки
Огузская группа
Огузо-туркменская подгруппа

Удины
Северокавказская надсемья
Нахско-дагестанская семья
Лезгинская группа

Удмурты
Уральская семья
Финно-угорская ветвь
Финно-пермская подветвь
Пермская группа

Удэгейцы
Алтайская семья
Тунгусо-маньчжурская ветвь

Узбеки
Тюркские языки
Карлукская ветвь
Карлукско-хорезмийская группа

Уйгуры
Тюркская семья
Карлукская ветвь
Карлукско-хорезмийская группа

Уйльта
Алтайская семья
Тунгусо-маньчжурская ветвь

Украинцы
Индоевропейская семья
Балто-славянская ветвь
Славянская группа
Восточнославянская подгруппа

Ульчи
Алтайская семья
Тунгусо-маньчжурская группа

Финны
Уральская семья
Финно-угорская ветвь
Финно-пермская подветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Финны-ингерманландцы
Уральская семья
Финно-угорская ветвь
Финно-пермская подветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Французы
Индоевропейские языки
Италийская ветвь
Романские языки
Галло-романские языки
Французский язык

Хакасы
Тюркские языки
Хакасская группа

Ханты
Уральская семья
Финно-угорская ветвь
Угорская подветвь
Обско-угорская группа

Хемшилы
Индоевропейская семья
Армянская языковая группа
Восточноармянский и западноармянский

Хорваты
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Западная подгруппа

Цахуры
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Лезгинская ветвь

Цыгане
Индоевропейская семья
Индоиранская ветвь
Индоарийская группа

Цыгане среднеазиатские
Индоевропейская семья
Индоиранская ветвь
Иранская группа
Юго-западная подгруппа

Черкесы
Северокавказские языки
Абхазо-адыгская семья
Адыгская группа

Черногорцы
Индоевропейская семья
Славянская ветвь
Южнославянская группа
Западная подгруппа

Чехи
Индоевропейская семья
Славянская ветвь
Западнославянская группа
Чешско-словацкая подгруппа

Чеченцы
Северокавказская надсемья (необщепризнано)
Нахско-дагестанская семья
Нахская ветвь
Вайнахская группа

Чеченцы-аккинцы
Северокавказская надсемья (не общепризнано)
Нахско-дагестанская семья
Нахская ветвь
Чеченский язык

Чуванцы
Чукотско-камчатская семья
Чукотско-корякская ветвь

Чуваши
Алтайские языки (спорно)
Тюркская ветвь
Булгарская группа

Чукчи
Чукотско-камчатская семья
Чукотско-корякская ветвь

Чулымцы
Алтайские языки (спорно)
Тюркская ветвь
Хакасская группа
Северноалтайская группа

Шапсуги
Северокавказская надсемья (необщепризнано)
Абхазо-адыгская семья
Адыгская группа

Шорцы
Алтайские языки (спорно)
Тюркская ветвь
Хакасская группа
Северноалтайская группа

Эвенки
Тунгусо-маньчжурская семья

Эвены
Алтайская семья
Тунгусо-маньчжурская группа

Энцы
Уральская семья
Самодийская ветвь
Северная группа

Эскимосы
Эскимосско-алеутские языки

Эстонцы
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Сету
Уральская семья
Финно-угорская ветвь
Финно-волжская группа
Прибалтийско-финская подгруппа

Юкагиры
Юкагирская семья

Якуты
Тюркские языки
Якутская ветвь

Японцы
Японо-рюкюские языки"""

'Абазины\nСеверокавказская надсемья (необщепризнано)\nАбхазо-адыгская семья\nАбхазо-абазинская ветвь языков\n\nАбхазы\nСеверокавказская надсемья (необщепризнано)\nАбхазо-адыгская семья\nАбхазо-абазинская группа\n\nАварцы\nСеверокавказская надсемья (необщепризнано)\nНахско-дагестанская семья\nАваро-андо-цезская ветвь\nАваро-андийская группа\n\nАндийцы\nСеверокавказская надсемья (необщепризнано)\nНахско-дагестанская семья\nАваро-андо-цезская ветвь\nАваро-андийская подветвь\nАндийская группа\n\nАрчинцы\nкавказские языки\nнахско-дагестанские языки\nдагестанские языки\nлезгинские языки\nарчинский язык\n\nАхвахцы\nНахско-дагестанская семья\nАваро-андо-цезская ветвь\nАваро-андийская подветвь\nАндийская группа\n\nБагулалы\nСеверокавказская надсемья (необщепризнано)\nНахско-дагестанская семья\nАваро-андо-цезская ветвь\nАваро-андийская подветвь\nАндийская подгруппа\n\nБежтинцы\nСеверокавказская семья (необщепризнано)\nНахско-дагестанская семья\nАваро-андо-цезская ветвь\nЦезская группа\n\nБотлихц

In [ ]:
'язык':{'макросемья':'',
'семья':'',
'подсемья':'',
'надветвь':'',
'зона':'',
'подзона':'',
'ветвь':'',
'подветвь':'',
'группа':'',
'подгруппа':'',
'подподгруппа':'',
'микрогруппа':''}

In [798]:
input()

Индоевропейская семья Славянская ветвь Восточнославянская группа


'Индоевропейская семья Славянская ветвь Восточнославянская группа'

In [ ]:
Универсальный классификатор языков

In [ ]:
{'':{'семья':'',
    'подсемья':,


In [ ]:
Индоевропейская семья
Славянская ветвь
Восточнославянская группа

# Графика

# Работа с картами

# Разработка библиотеки

In [ ]:
class Region(pd.DataFrame):
    